# Tumor Segmentation Demo
## BME 271D - Ege, Max, Sasha

Click Runtime > Run All to see results!

In [ ]:
!pip install -q numpy matplotlib scipy scikit-image pandas
print('Packages installed')

In [ ]:
import tumor_segmentation as ts
import matplotlib.pyplot as plt
print('Ready!')

In [ ]:
image = ts.load_grayscale_image('data/images/tumor_001.png')
mask = ts.load_binary_mask('data/masks/tumor_001.png')

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].imshow(image, cmap='gray')
axes[0].set_title('Tumor Image')
axes[0].axis('off')

axes[1].imshow(image, cmap='gray')
axes[1].imshow(mask, cmap='Reds', alpha=0.5)
axes[1].set_title('Ground Truth')
axes[1].axis('off')
plt.show()

In [ ]:
F_shift, mag = ts.compute_fft_spectrum(image)
fig = ts.visualize_frequency_spectrum(image, F_shift)
plt.show()

In [ ]:
params = {'hp_radius': 25, 'bp_r1': 10, 'bp_r2': 40, 'canny_sigma': 1.0, 'gaussian_sigma': 1.0}
results = ts.run_single_image_experiment(image, mask, params, verbose=True)

print('\n=== RESULTS ===')
for method, data in results.items():
    print(f"{method}: Dice = {data['metrics']['dice']:.3f}")

In [ ]:
masks_dict = {name: data['mask'] for name, data in results.items()}
fig = ts.plot_segmentation_comparison(image, masks_dict, mask)
plt.show()